## **Ⅰ. 부동산 매매 정보**

In [1]:
import datetime
import pandas as pd

### **1. 필요한 column만 선택하기**
**① 수집 기간: 2015년 6월 ~ 2020년 5월** (이동평균을 계산하기 위해 15년 3, 4, 5월 추가 수집)<br>
　 **주거 유형: 오피스텔, 아파트, 연립다세대, 단독다가구**

In [2]:
period = pd.date_range('2015-03-01', periods=63, freq='MS')
period

DatetimeIndex(['2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01',
               '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01',
               '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01',
               '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01',
               '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01',
               '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01',
               '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
               '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
               '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
               '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
               '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
      

In [3]:
types = ['오피스텔(매매)', '아파트(매매)', '연립다세대(매매)', '단독다가구(매매)']
types

['오피스텔(매매)', '아파트(매매)', '연립다세대(매매)', '단독다가구(매매)']

**② 월별로 나뉘어져 있는 데이터를 합치기**

In [ ]:
for t in types:
    data_df = pd.DataFrame()
    
    for date in period:
        date = datetime.datetime.strftime(date, '%Y%m')

        fPath = 'C:/Users/671/Downloads/{0}_실거래가_{1}.xlsx'.format(t, date)
        
        raw = pd.read_excel(fPath, encoding='utf-8')
        raw = raw[15:]
        raw.rename(columns=raw.iloc[0], inplace=True)
        raw.drop(raw.index[0], inplace=True)
        raw.reset_index(drop=True, inplace=True)
        raw = raw[['시군구', '거래금액(만원)']]
        raw['시도'] = raw['시군구'].apply(lambda x: x.split()[0])
        raw['거래금액(만원)'] = raw['거래금액(만원)'].apply(lambda x: int(x.replace(',', '').strip()))
        
        avgprice = raw.groupby(['시도']).mean()
        avgprice.reset_index(inplace=True)
        
        count = raw.groupby(['시도']).count()
        count.drop(['거래금액(만원)'], axis=1, inplace=True)
        count.reset_index(inplace=True)
        
        data = pd.merge(count, avgprice, how='left', on='시도')
        data.columns = [['시도', '실거래량', '매매가평균']]
        data['거래년도'] = date[:4]
        data['거래월'] = date[4:]
        data = data[['시도', '거래년도', '거래월', '실거래량', '매매가평균']]
        
        data_df = data_df.append(data)

    finalPath = './data/{0}.csv'.format(t)
    data_df.to_csv(finalPath, encoding='utf-8-sig', header=True, index=False)

### **2. 주거타입별 '전국' 거래량, 평균 매매량과 매매가의 이동평균 계산**
**① 거래년도, 거래월을 기준으로 groupby 후, '전국' 실거래량과 매매가 평균 계산<br>
② 매매가의 이동평균 계산하기<br>
③ 주거타입별로 분리되어 있는 데이터를 하나의 데이터프레임으로 합치기**

In [4]:
data = pd.DataFrame()

#### **2-1) 오피스텔**

In [5]:
officetel = pd.read_csv('./data/{}.csv'.format(types[0]), encoding='utf-8-sig')

print(officetel.shape)
officetel.head()

(1069, 5)


,시도,거래년도,거래월,실거래량,매매가평균
0,강원도,2015,3,22,2938.636364
1,경기도,2015,3,1330,14606.374436
2,경상남도,2015,3,160,9997.431250
3,경상북도,2015,3,51,9011.686275
4,광주광역시,2015,3,38,6171.815789


In [6]:
total_count = officetel[['거래년도', '거래월', '실거래량']].groupby(['거래년도', '거래월']).sum()
total_count.reset_index(inplace=True)

total_avg = officetel[['거래년도', '거래월', '매매가평균']].groupby(['거래년도', '거래월']).mean()
total_avg.reset_index(inplace=True)

officetel_total = pd.merge(total_count, total_avg, how='outer')
officetel_total['시도'] = '전국'

officetel_total['매매가 이동평균'] = officetel_total['매매가평균'].rolling(window=3).mean()

officetel_total = officetel_total[['시도', '거래년도', '거래월', '실거래량', '매매가평균', '매매가 이동평균']]
print(officetel_total.shape)        
officetel_total.head()

(63, 6)


,시도,거래년도,거래월,실거래량,매매가평균,매매가 이동평균
0,전국,2015,3,4174,10283.710005,NaN
1,전국,2015,4,3861,10754.518446,NaN
2,전국,2015,5,3564,10299.137831,10445.788761
3,전국,2015,6,3642,10843.960595,10632.538957
4,전국,2015,7,3705,10951.903010,10698.333812


In [7]:
data = data.append(officetel_total)
data.set_index(['시도', '거래년도', '거래월'], inplace=True)
data

실거래량         매매가평균      매매가 이동평균
시도 거래년도 거래월                                  
전국 2015 3    4174  10283.710005           NaN
        4    3861  10754.518446           NaN
        5    3564  10299.137831  10445.788761
        6    3642  10843.960595  10632.538957
        7    3705  10951.903010  10698.333812
...           ...           ...           ...
   2020 1    3135  13933.843678  13985.761173
        2    3637  12891.227380  13490.118799
        3    2736  13340.218213  13388.429757
        4    2886  13459.060450  13230.168681
        5    3012  14311.204844  13703.494502

[63 rows x 3 columns]

In [8]:
data.columns = [['오피스텔' for i in range(3)], ['실거래량', '매매가평균', '매매가 이동평균']]
data.head()

오피스텔                            
             실거래량         매매가평균      매매가 이동평균
시도 거래년도 거래월                                  
전국 2015 3    4174  10283.710005           NaN
        4    3861  10754.518446           NaN
        5    3564  10299.137831  10445.788761
        6    3642  10843.960595  10632.538957
        7    3705  10951.903010  10698.333812

#### **2-2) 아파트**

In [10]:
apart = pd.read_csv('./data/{}.csv'.format(types[1]), encoding='utf-8-sig')

print(apart.shape)
apart.head()

(1071, 5)


,시도,거래년도,거래월,실거래량,매매가평균
0,강원도,2015,3,2301,12239.431117
1,경기도,2015,3,25272,27155.648109
2,경상남도,2015,3,5672,18047.775388
3,경상북도,2015,3,3408,14216.422535
4,광주광역시,2015,3,2959,16461.169652


In [11]:
total_count = apart[['거래년도', '거래월', '실거래량']].groupby(['거래년도', '거래월']).sum()
total_count.reset_index(inplace=True)

total_avg = apart[['거래년도', '거래월', '매매가평균']].groupby(['거래년도', '거래월']).mean()
total_avg.reset_index(inplace=True)

apart_total = pd.merge(total_count, total_avg, how='outer')
apart_total['시도'] = '전국'

apart_total['매매가 이동평균'] = apart_total['매매가평균'].rolling(window=3).mean()

apart_total = apart_total[['시도', '거래년도', '거래월', '실거래량', '매매가평균', '매매가 이동평균']]        
print(apart_total.shape)        
apart_total.head()

(63, 6)


,시도,거래년도,거래월,실거래량,매매가평균,매매가 이동평균
0,전국,2015,3,87137,19764.579530,NaN
1,전국,2015,4,71654,19982.582043,NaN
2,전국,2015,5,60953,20094.379419,19947.180331
3,전국,2015,6,62737,20290.322242,20122.427901
4,전국,2015,7,61775,20536.351868,20307.017843


In [12]:
apart_total.set_index(['시도', '거래년도', '거래월'], inplace=True)
data = pd.merge(data, apart_total, left_index=True, right_index=True, how='outer')
data

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


(오피스텔, 실거래량)  (오피스텔, 매매가평균)  (오피스텔, 매매가 이동평균)   실거래량  \
시도 거래년도 거래월                                                         
전국 2015 3            4174   10283.710005               NaN  87137   
        4            3861   10754.518446               NaN  71654   
        5            3564   10299.137831      10445.788761  60953   
        6            3642   10843.960595      10632.538957  62737   
        7            3705   10951.903010      10698.333812  61775   
...                   ...            ...               ...    ...   
   2020 1            3135   13933.843678      13985.761173  58584   
        2            3637   12891.227380      13490.118799  81831   
        3            2736   13340.218213      13388.429757  50498   
        4            2886   13459.060450      13230.168681  43971   
        5            3012   14311.204844      13703.494502  60340   

                    매매가평균      매매가 이동평균  
시도 거래년도 거래월                              
전국 2015 3    19764.579530           NaN  
        4    19982.582043           NaN  
        5    20094.379419  19947.180331  
        6    20290.322242  20122.427901  
        7    20536.351868  20307.017843  
...                   ...           ...  
   2020 1    26996.540215  28410.706027  
        2    27340.402246  27613.900908  
        3    25204.100298  26513.680920  
        4    25346.753291  25963.751945  
        5    27811.559261  26120.804283  

[63 rows x 6 columns]

In [13]:
data.columns = [['오피스텔' for i in range(3)] + ['아파트' for i in range(3)],
                ['실거래량', '매매가평균', '매매가 이동평균'] * 2]
data

오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 3    4174  10283.710005           NaN  87137  19764.579530   
        4    3861  10754.518446           NaN  71654  19982.582043   
        5    3564  10299.137831  10445.788761  60953  20094.379419   
        6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   
...           ...           ...           ...    ...           ...   
   2020 1    3135  13933.843678  13985.761173  58584  26996.540215   
        2    3637  12891.227380  13490.118799  81831  27340.402246   
        3    2736  13340.218213  13388.429757  50498  25204.100298   
        4    2886  13459.060450  13230.168681  43971  25346.753291   
        5    3012  14311.204844  13703.494502  60340  27811.559261   

                           
                 매매가 이동평균  
시도 거래년도 거래월                
전국 2015 3             NaN  
        4             NaN  
        5    19947.180331  
        6    20122.427901  
        7    20307.017843  
...                   ...  
   2020 1    28410.706027  
        2    27613.900908  
        3    26513.680920  
        4    25963.751945  
        5    26120.804283  

[63 rows x 6 columns]

#### **2-3) 연립다세대**

In [14]:
multiplex = pd.read_csv('./data/{}.csv'.format(types[2]), encoding='utf-8')

print(multiplex.shape)
multiplex.head()

(1070, 5)


,시도,거래년도,거래월,실거래량,매매가평균
0,강원도,2015,3,229,6493.218341
1,경기도,2015,3,5959,13234.291660
2,경상남도,2015,3,537,10747.564246
3,경상북도,2015,3,716,8791.149441
4,광주광역시,2015,3,154,7756.461039


In [15]:
total_count = multiplex[['거래년도', '거래월', '실거래량']].groupby(['거래년도', '거래월']).sum()
total_count.reset_index(inplace=True)

total_avg = multiplex[['거래년도', '거래월', '매매가평균']].groupby(['거래년도', '거래월']).mean()
total_avg.reset_index(inplace=True)

multiplex_total = pd.merge(total_count, total_avg, how='outer')
multiplex_total['시도'] = '전국'

multiplex_total['매매가 이동평균'] = multiplex_total['매매가평균'].rolling(window=3).mean()

multiplex_total = multiplex_total[['시도', '거래년도', '거래월', '실거래량', '매매가평균', '매매가 이동평균']]
print(multiplex_total.shape)
multiplex_total.head()

(63, 6)


,시도,거래년도,거래월,실거래량,매매가평균,매매가 이동평균
0,전국,2015,3,20757,10062.448196,NaN
1,전국,2015,4,19247,9954.501596,NaN
2,전국,2015,5,16458,10233.163889,10083.371227
3,전국,2015,6,16595,10154.538444,10114.067976
4,전국,2015,7,16910,10480.719940,10289.474091


In [16]:
multiplex_total.set_index(['시도', '거래년도', '거래월'], inplace=True)
data = pd.merge(data, multiplex_total, left_index=True, right_index=True, how='outer')
data

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


(오피스텔, 실거래량)  (오피스텔, 매매가평균)  (오피스텔, 매매가 이동평균)  (아파트, 실거래량)  \
시도 거래년도 거래월                                                               
전국 2015 3            4174   10283.710005               NaN        87137   
        4            3861   10754.518446               NaN        71654   
        5            3564   10299.137831      10445.788761        60953   
        6            3642   10843.960595      10632.538957        62737   
        7            3705   10951.903010      10698.333812        61775   
...                   ...            ...               ...          ...   
   2020 1            3135   13933.843678      13985.761173        58584   
        2            3637   12891.227380      13490.118799        81831   
        3            2736   13340.218213      13388.429757        50498   
        4            2886   13459.060450      13230.168681        43971   
        5            3012   14311.204844      13703.494502        60340   

             (아파트, 매매가평균)  (아파트, 매매가 이동평균)   실거래량         매매가평균      매매가 이동평균  
시도 거래년도 거래월                                                                    
전국 2015 3    19764.579530              NaN  20757  10062.448196           NaN  
        4    19982.582043              NaN  19247   9954.501596           NaN  
        5    20094.379419     19947.180331  16458  10233.163889  10083.371227  
        6    20290.322242     20122.427901  16595  10154.538444  10114.067976  
        7    20536.351868     20307.017843  16910  10480.719940  10289.474091  
...                   ...              ...    ...           ...           ...  
   2020 1    26996.540215     28410.706027  12066  13353.044049  13431.037429  
        2    27340.402246     27613.900908  15422  13206.772830  13438.229863  
        3    25204.100298     26513.680920  12682  13571.211669  13377.009516  
        4    25346.753291     25963.751945  12751  14418.880101  13732.288200  
        5    27811.559261     26120.804283  15882  13937.813686  13975.968485  

[63 rows x 9 columns]

In [17]:
data.columns = [['오피스텔' for i in range(3)] + ['아파트' for i in range(3)] + ['연립다세대' for i in range(3)],
                ['실거래량', '매매가평균', '매매가 이동평균'] * 3]
data

오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 3    4174  10283.710005           NaN  87137  19764.579530   
        4    3861  10754.518446           NaN  71654  19982.582043   
        5    3564  10299.137831  10445.788761  60953  20094.379419   
        6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   
...           ...           ...           ...    ...           ...   
   2020 1    3135  13933.843678  13985.761173  58584  26996.540215   
        2    3637  12891.227380  13490.118799  81831  27340.402246   
        3    2736  13340.218213  13388.429757  50498  25204.100298   
        4    2886  13459.060450  13230.168681  43971  25346.753291   
        5    3012  14311.204844  13703.494502  60340  27811.559261   

                           연립다세대                              
                 매매가 이동평균   실거래량         매매가평균      매매가 이동평균  
시도 거래년도 거래월                                                   
전국 2015 3             NaN  20757  10062.448196           NaN  
        4             NaN  19247   9954.501596           NaN  
        5    19947.180331  16458  10233.163889  10083.371227  
        6    20122.427901  16595  10154.538444  10114.067976  
        7    20307.017843  16910  10480.719940  10289.474091  
...                   ...    ...           ...           ...  
   2020 1    28410.706027  12066  13353.044049  13431.037429  
        2    27613.900908  15422  13206.772830  13438.229863  
        3    26513.680920  12682  13571.211669  13377.009516  
        4    25963.751945  12751  14418.880101  13732.288200  
        5    26120.804283  15882  13937.813686  13975.968485  

[63 rows x 9 columns]

#### **2-4) 단독다가구**

In [18]:
single = pd.read_csv('./data/{}.csv'.format(types[3]), encoding='utf-8-sig')

print(single.shape)
single.head()

(1071, 5)


,시도,거래년도,거래월,실거래량,매매가평균
0,강원도,2015,3,467,19207.169165
1,경기도,2015,3,1693,47411.096279
2,경상남도,2015,3,1252,26232.453674
3,경상북도,2015,3,1191,24789.150294
4,광주광역시,2015,3,585,23549.252991


In [19]:
total_count = single[['거래년도', '거래월', '실거래량']].groupby(['거래년도', '거래월']).sum()
total_count.reset_index(inplace=True)

total_avg = single[['거래년도', '거래월', '매매가평균']].groupby(['거래년도', '거래월']).mean()
total_avg.reset_index(inplace=True)

single_total = pd.merge(total_count, total_avg, how='outer')
single_total['시도'] = '전국'

single_total['매매가 이동평균'] = single_total['매매가평균'].rolling(window=3).mean()

single_total = single_total[['시도', '거래년도', '거래월', '실거래량', '매매가평균', '매매가 이동평균']]
print(single_total.shape)
single_total.head()

(63, 6)


,시도,거래년도,거래월,실거래량,매매가평균,매매가 이동평균
0,전국,2015,3,13094,31548.478864,NaN
1,전국,2015,4,13072,30989.082308,NaN
2,전국,2015,5,12324,31803.233129,31446.931434
3,전국,2015,6,12538,32745.757088,31846.024175
4,전국,2015,7,11949,32281.283400,32276.757872


In [20]:
single_total.set_index(['시도', '거래년도', '거래월'], inplace=True)
data = pd.merge(data, single_total, left_index=True, right_index=True, how='outer')
data

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


(오피스텔, 실거래량)  (오피스텔, 매매가평균)  (오피스텔, 매매가 이동평균)  (아파트, 실거래량)  \
시도 거래년도 거래월                                                               
전국 2015 3            4174   10283.710005               NaN        87137   
        4            3861   10754.518446               NaN        71654   
        5            3564   10299.137831      10445.788761        60953   
        6            3642   10843.960595      10632.538957        62737   
        7            3705   10951.903010      10698.333812        61775   
...                   ...            ...               ...          ...   
   2020 1            3135   13933.843678      13985.761173        58584   
        2            3637   12891.227380      13490.118799        81831   
        3            2736   13340.218213      13388.429757        50498   
        4            2886   13459.060450      13230.168681        43971   
        5            3012   14311.204844      13703.494502        60340   

             (아파트, 매매가평균)  (아파트, 매매가 이동평균)  (연립다세대, 실거래량)  (연립다세대, 매매가평균)  \
시도 거래년도 거래월                                                                 
전국 2015 3    19764.579530              NaN          20757    10062.448196   
        4    19982.582043              NaN          19247     9954.501596   
        5    20094.379419     19947.180331          16458    10233.163889   
        6    20290.322242     20122.427901          16595    10154.538444   
        7    20536.351868     20307.017843          16910    10480.719940   
...                   ...              ...            ...             ...   
   2020 1    26996.540215     28410.706027          12066    13353.044049   
        2    27340.402246     27613.900908          15422    13206.772830   
        3    25204.100298     26513.680920          12682    13571.211669   
        4    25346.753291     25963.751945          12751    14418.880101   
        5    27811.559261     26120.804283          15882    13937.813686   

             (연립다세대, 매매가 이동평균)   실거래량         매매가평균      매매가 이동평균  
시도 거래년도 거래월                                                        
전국 2015 3                  NaN  13094  31548.478864           NaN  
        4                  NaN  13072  30989.082308           NaN  
        5         10083.371227  12324  31803.233129  31446.931434  
        6         10114.067976  12538  32745.757088  31846.024175  
        7         10289.474091  11949  32281.283400  32276.757872  
...                        ...    ...           ...           ...  
   2020 1         13431.037429   6318  36730.896190  37935.471889  
        2         13438.229863   7496  40461.782053  38511.994225  
        3         13377.009516   6938  35882.110215  37691.596153  
        4         13732.288200   6990  36329.302826  37557.731698  
        5         13975.968485   8051  36961.931013  36391.114685  

[63 rows x 12 columns]

In [21]:
data.columns = [['오피스텔' for i in range(3)] + ['아파트' for i in range(3)] +
                ['연립다세대' for i in range(3)] + ['단독다가구' for i in range(3)],
                ['실거래량', '매매가평균', '매매가 이동평균'] * 4]
data

오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 3    4174  10283.710005           NaN  87137  19764.579530   
        4    3861  10754.518446           NaN  71654  19982.582043   
        5    3564  10299.137831  10445.788761  60953  20094.379419   
        6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   
...           ...           ...           ...    ...           ...   
   2020 1    3135  13933.843678  13985.761173  58584  26996.540215   
        2    3637  12891.227380  13490.118799  81831  27340.402246   
        3    2736  13340.218213  13388.429757  50498  25204.100298   
        4    2886  13459.060450  13230.168681  43971  25346.753291   
        5    3012  14311.204844  13703.494502  60340  27811.559261   

                           연립다세대                              단독다가구  \
                 매매가 이동평균   실거래량         매매가평균      매매가 이동평균   실거래량   
시도 거래년도 거래월                                                           
전국 2015 3             NaN  20757  10062.448196           NaN  13094   
        4             NaN  19247   9954.501596           NaN  13072   
        5    19947.180331  16458  10233.163889  10083.371227  12324   
        6    20122.427901  16595  10154.538444  10114.067976  12538   
        7    20307.017843  16910  10480.719940  10289.474091  11949   
...                   ...    ...           ...           ...    ...   
   2020 1    28410.706027  12066  13353.044049  13431.037429   6318   
        2    27613.900908  15422  13206.772830  13438.229863   7496   
        3    26513.680920  12682  13571.211669  13377.009516   6938   
        4    25963.751945  12751  14418.880101  13732.288200   6990   
        5    26120.804283  15882  13937.813686  13975.968485   8051   

                                         
                    매매가평균      매매가 이동평균  
시도 거래년도 거래월                              
전국 2015 3    31548.478864           NaN  
        4    30989.082308           NaN  
        5    31803.233129  31446.931434  
        6    32745.757088  31846.024175  
        7    32281.283400  32276.757872  
...                   ...           ...  
   2020 1    36730.896190  37935.471889  
        2    40461.782053  38511.994225  
        3    35882.110215  37691.596153  
        4    36329.302826  37557.731698  
        5    36961.931013  36391.114685  

[63 rows x 12 columns]

In [22]:
data.to_csv('./data/월별 전국 매매정보.csv', encoding='utf-8-sig', header=True, index=True)

### **3. 주거형태별 데이터 총합**

In [24]:
data['실거래량'] = data['오피스텔', '실거래량'] + data['아파트', '실거래량'] + data['연립다세대', '실거래량'] + data['단독다가구', '실거래량']
data['매매가평균'] = (data['오피스텔', '매매가평균'] + data['아파트', '매매가평균'] + data['연립다세대', '매매가평균'] + data['단독다가구', '매매가평균']) / 4
data['매매가 이동평균'] = data['매매가평균'].rolling(window=3).mean()

print(data.shape)
data.head()

(63, 15)


오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 3    4174  10283.710005           NaN  87137  19764.579530   
        4    3861  10754.518446           NaN  71654  19982.582043   
        5    3564  10299.137831  10445.788761  60953  20094.379419   
        6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   

                           연립다세대                              단독다가구  \
                 매매가 이동평균   실거래량         매매가평균      매매가 이동평균   실거래량   
시도 거래년도 거래월                                                           
전국 2015 3             NaN  20757  10062.448196           NaN  13094   
        4             NaN  19247   9954.501596           NaN  13072   
        5    19947.180331  16458  10233.163889  10083.371227  12324   
        6    20122.427901  16595  10154.538444  10114.067976  12538   
        7    20307.017843  16910  10480.719940  10289.474091  11949   

                                           실거래량         매매가평균      매매가 이동평균  
                    매매가평균      매매가 이동평균                                      
시도 거래년도 거래월                                                                  
전국 2015 3    31548.478864           NaN  125162  17914.804149           NaN  
        4    30989.082308           NaN  107834  17920.171098           NaN  
        5    31803.233129  31446.931434   93299  18107.478567  17980.817938  
        6    32745.757088  31846.024175   95512  18508.644592  18178.764753  
        7    32281.283400  32276.757872   94339  18562.564554  18392.895905

In [25]:
data.columns = [['오피스텔' for i in range(3)] + ['아파트' for i in range(3)] +
                ['연립다세대' for i in range(3)] + ['단독다가구' for i in range(3)] + ['전체' for i in range(3)],
                ['실거래량', '매매가평균', '매매가 이동평균'] * 5]
data

오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 3    4174  10283.710005           NaN  87137  19764.579530   
        4    3861  10754.518446           NaN  71654  19982.582043   
        5    3564  10299.137831  10445.788761  60953  20094.379419   
        6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   
...           ...           ...           ...    ...           ...   
   2020 1    3135  13933.843678  13985.761173  58584  26996.540215   
        2    3637  12891.227380  13490.118799  81831  27340.402246   
        3    2736  13340.218213  13388.429757  50498  25204.100298   
        4    2886  13459.060450  13230.168681  43971  25346.753291   
        5    3012  14311.204844  13703.494502  60340  27811.559261   

                           연립다세대                              단독다가구  \
                 매매가 이동평균   실거래량         매매가평균      매매가 이동평균   실거래량   
시도 거래년도 거래월                                                           
전국 2015 3             NaN  20757  10062.448196           NaN  13094   
        4             NaN  19247   9954.501596           NaN  13072   
        5    19947.180331  16458  10233.163889  10083.371227  12324   
        6    20122.427901  16595  10154.538444  10114.067976  12538   
        7    20307.017843  16910  10480.719940  10289.474091  11949   
...                   ...    ...           ...           ...    ...   
   2020 1    28410.706027  12066  13353.044049  13431.037429   6318   
        2    27613.900908  15422  13206.772830  13438.229863   7496   
        3    26513.680920  12682  13571.211669  13377.009516   6938   
        4    25963.751945  12751  14418.880101  13732.288200   6990   
        5    26120.804283  15882  13937.813686  13975.968485   8051   

                                             전체                              
                    매매가평균      매매가 이동평균    실거래량         매매가평균      매매가 이동평균  
시도 거래년도 거래월                                                                  
전국 2015 3    31548.478864           NaN  125162  17914.804149           NaN  
        4    30989.082308           NaN  107834  17920.171098           NaN  
        5    31803.233129  31446.931434   93299  18107.478567  17980.817938  
        6    32745.757088  31846.024175   95512  18508.644592  18178.764753  
        7    32281.283400  32276.757872   94339  18562.564554  18392.895905  
...                   ...           ...     ...           ...           ...  
   2020 1    36730.896190  37935.471889   80103  22753.581033  23440.744129  
        2    40461.782053  38511.994225  108386  23475.046127  23263.560949  
        3    35882.110215  37691.596153   72854  21999.410099  22742.679086  
        4    36329.302826  37557.731698   66598  22388.499167  22620.985131  
        5    36961.931013  36391.114685   87285  23255.627201  22547.845489  

[63 rows x 15 columns]

### **4. 수집기간(2017.06 ~ 2020.05)에 맞춰 데이터 정리하기**
**① 수집기간 mask 설정하여 데이터 추출하기**

In [26]:
mask = (data.index.get_level_values('거래년도') == 2015) & (data.index.get_level_values('거래월').isin([3, 4, 5]))
data = data.loc[~mask]
data

오피스텔                                아파트                \
             실거래량         매매가평균      매매가 이동평균   실거래량         매매가평균   
시도 거래년도 거래월                                                          
전국 2015 6    3642  10843.960595  10632.538957  62737  20290.322242   
        7    3705  10951.903010  10698.333812  61775  20536.351868   
        8    2885  11380.385622  11058.749742  52133  20388.572223   
        9    3608  12018.864568  11450.384400  55654  20956.946706   
        10   3876  11757.431819  11718.894003  64446  21431.030895   
        11   3446  11168.539618  11648.278669  47218  21627.311525   
        12   3357  11481.881616  11469.284351  36855  21166.739025   
   2016 1    2802  10717.963204  11122.794813  35629  20792.172795   
        2    2690  11159.829028  11119.891283  36424  20541.233757   
        3    4039  10914.443433  10930.745221  53434  20086.315871   
        4    3478  11204.878601  11093.050354  53500  20759.187419   
        5    3606  10979.591652  11032.971228  51356  20928.633699   
        6    4072  11675.982606  11286.817620  59964  21428.923444   
        7    3745  11362.379973  11339.318077  59136  21792.643066   
        8    3355  11572.119481  11536.827353  59811  22118.393717   
        9    3332  12368.068687  11767.522714  60995  22806.520325   
        10   3912  11794.486264  11911.558144  71670  23048.897245   
        11   3278  11734.382981  11965.645977  46722  22544.258492   
        12   2906  11842.437059  11790.435434  36191  22627.429536   
   2017 1    2681  11086.896868  11554.572303  30126  22005.787000   
        2    3495  11118.893120  11349.409016  41852  21900.447106   
        3    4132  11414.437573  11206.742520  51241  21916.994149   
        4    3098  11569.243268  11367.524654  47684  22198.857740   
        5    3278  11156.377306  11380.019382  58833  23190.944034   
        6    3740  11667.758301  11464.459625  62631  23407.903385   
        7    3458  11845.038797  11556.391468  61708  24212.505330   
        8    2725  11481.165857  11664.654319  45227  23033.883459   
        9    2897  11099.860114  11475.354923  47247  24084.083522   
        10   2325  11902.349425  11494.458465  39636  24090.753352   
        11   2701  11312.070964  11438.093501  43933  24472.993449   
        12   2478  12328.230229  11847.550206  40193  25171.580691   
   2018 1    3122  11974.512340  11871.604511  50677  24911.293619   
        2    2660  11935.554925  12079.432498  43502  24411.583565   
        3    3617  11762.227402  11890.764889  55215  23879.310162   
        4    2698  11420.417384  11706.066570  35665  23239.195092   
        5    2592  11788.313712  11656.986166  36816  23057.690126   
        6    2666  12173.400259  11794.043785  38781  23412.522430   
        7    2315  11705.290255  11889.001409  36512  24206.379452   
        8    2737  12745.844805  12208.178439  56304  25397.833354   
        9    3092  11830.479692  12093.871584  50856  25887.508532   
        10   2831  12133.915123  12236.746540  43985  24722.292797   
        11   2179  12880.251833  12281.548882  30734  24207.729589   
        12   2184  12863.923112  12626.030022  26861  23275.124622   
   2019 1    2295  11773.504846  12505.893263  28401  23307.625969   
        2    2032  12816.885084  12484.771014  26132  22948.926868   
        3    2597  12035.626603  12208.672177  33394  23359.455549   
        4    2465  12076.688160  12309.733282  32308  24302.730620   
        5    2480  11809.464466  11973.926410  35096  25233.821696   
        6    2404  12248.290812  12044.814480  38373  26122.075125   
        7    2570  13248.527503  12435.427594  45213  26812.124969   
        8    3285  13175.136377  12890.651564  41944  26462.116860   
        9    2930  13598.479028  13340.714303  44025  27507.809275   
        10   3783  13071.723072  13281.779492  64689  28250.628169   
        11   3721  14378.154504  13682.785535  74305  29730.817601   
        12   4271  1364

**② 주거유형 데이터를 통합한 '전체' 데이터만 남기기**

In [36]:
data = data['전체']

print(data.shape)
data.head()

(60, 3)


실거래량         매매가평균      매매가 이동평균
시도 거래년도 거래월                                   
전국 2015 6    95512  18508.644592  18178.764753
        7    94339  18562.564554  18392.895905
        8    79605  18846.474674  18639.227940
        9    85899  18833.881894  18747.640374
        10   98098  19121.626182  18933.994250

In [37]:
data.to_csv('./data/월별 전국 매매정보 최종.csv', encoding='utf-8-sig', header=True, index=True)

---

## **Ⅱ. 부동산 소비심리지수**

In [28]:
fPath = './data/부동산시장_소비심리지수_부동산시장.xlsx'
realestate_idx = pd.read_excel(fPath, encoding='utf-8')

realestate_idx = realestate_idx[9:]
realestate_idx.rename(columns=realestate_idx.iloc[0], inplace=True)
realestate_idx.drop(realestate_idx.index[0], inplace=True)
realestate_idx.reset_index(drop=True, inplace=True)
realestate_idx.drop(realestate_idx.index[1], inplace=True)
realestate_idx.reset_index(drop=True, inplace=True)

print(realestate_idx.shape)
realestate_idx.head()

(18, 61)


,지 역,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,...,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월
0,전국,123.5,129.4,128.7,126.4,129.8,121.6,107.9,112.3,114.3,...,103.5,108.4,112.7,114.4,114.3,113,115.9,107,103.1,109.8
1,서울,129.7,138.9,136.2,131.5,138.1,128.5,110.7,117.6,120.1,...,118.2,122.5,130.6,129.7,125.6,116.5,120.1,109.8,102.2,112.5
2,부산,126,131,130.3,127.9,134.7,127.2,112.7,116.7,119.5,...,92.6,100.4,101.5,113.3,110.4,105.8,105.9,99.5,97.5,101.8
3,대구,131.9,138.5,133.4,127.8,122.5,110.4,93.9,91.5,93.8,...,111.5,113.5,116,116.2,120.1,116.3,116,105,101.6,109.3
4,인천,123.4,130,130.3,129,131.9,123.1,108.1,114.8,116,...,103.8,110.6,115.7,112.8,110.7,114.5,123.2,117.4,109.7,115


In [29]:
realestate_idx = realestate_idx.drop(realestate_idx.index[1:])
realestate_idx

,지 역,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,...,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월
0,전국,123.5,129.4,128.7,126.4,129.8,121.6,107.9,112.3,114.3,...,103.5,108.4,112.7,114.4,114.3,113,115.9,107,103.1,109.8


### **1. 부동산 실거래 정보 데이터에 월별 부동산 소비 심리지수 표시하기**
**① 데이터프레임 합치기 (merge)**

In [30]:
period = pd.date_range('2015-06-01', periods=60, freq='MS')
period

DatetimeIndex(['2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01',
               '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01',
               '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01',
               '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01',
               '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01',
               '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01',
               '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01',
               '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01',
               '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01',
               '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01',
               '2018-10-01', '2018-11-01', '2018-12-01', '2019-01-01',
               '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01',
               '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
               '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01',
      

In [38]:
for date in period:
    year = datetime.datetime.strftime(date, '%Y')
    month = datetime.datetime.strftime(date, '%m')
    
    col_name = '{}년 {}월'.format(year, month)
    
    data.loc[(data.index.get_level_values('거래년도') == int(year)) & (data.index.get_level_values('거래월') == int(month)), '부동산 소비심리지수'] = \
    realestate_idx[col_name].tolist()
    
data

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


실거래량         매매가평균      매매가 이동평균  부동산 소비심리지수
시도 거래년도 거래월                                                
전국 2015 6     95512  18508.644592  18178.764753       123.5
        7     94339  18562.564554  18392.895905       129.4
        8     79605  18846.474674  18639.227940       128.7
        9     85899  18833.881894  18747.640374       126.4
        10    98098  19121.626182  18933.994250       129.8
        11    74611  19233.375577  19062.961217       121.6
        12    60660  19302.438871  19219.146877       107.9
   2016 1     56842  18823.764930  19119.859793       112.3
        2     56996  19252.789825  19126.331209       114.3
        3     86968  18517.710453  18864.755069       114.7
        4     85217  18786.990451  18852.496910       116.6
        5     81844  19448.881673  18917.860859       116.6
        6     92883  19870.781109  19368.884411       116.1
        7     89513  20024.716638  19781.459807       119.1
        8     89795  20206.450285  20033.982677       121.9
        9     91010  20789.200480  20340.122468       122.1
        10   106265  20534.664826  20510.105197       123.2
        11    73870  20793.171760  20705.679022       113.2
        12    59634  20578.717304  20635.517963       104.3
   2017 1     49379  20302.262840  20558.050635       107.4
        2     67943  19939.874300  20273.618148       111.2
        3     83727  19606.026999  19949.388046       111.5
        4     76222  19969.263510  19838.388269       112.1
        5     89765  20384.526821  19986.605777       115.2
        6     94983  20880.596808  20411.462380       114.2
        7     91556  21353.839358  20872.987662       118.1
        8     70537  20380.512955  20871.649707       112.1
        9     73661  20859.242831  20864.531715       110.5
        10    60648  21010.942897  20750.232894       109.9
        11    66891  21569.423034  21146.536254       106.4
        12    62117  22343.628094  21641.331342       102.2
   2018 1     73901  21306.521033  21739.857387       109.0
        2     64155  20815.412747  21488.520625       110.0
        3     85479  20410.318190  20844.083990       106.0
        4     59468  20315.593144  20513.774694       100.2
        5     60556  20354.120733  20360.010689        99.0
        6     61675  20528.790988  20399.501622        96.3
        7     57675  21124.152200  20669.021307        98.3
        8     80406  21488.852341  21047.265176       106.7
        9     74525  21517.272981  21376.759174       110.0
        10    66903  20619.234272  21208.453198       103.5
        11    49708  21110.817182  21082.441478        96.1
        12    45636  21741.155393  21157.068949        90.7
   2019 1     45619  21277.940208  21376.637594        90.0
        2     41326  20575.933702  21198.343101        92.7
        3     53908  20753.010540  20868.961483        90.9
        4     51948  20754.567786  20694.504009        87.7
        5     55519  21461.390579  20989.656302        91.3
        6     58284  22182.277574  21466.078647        97.9
        7     66007  22613.733674  22085.800609       101.7
        8     62741  22276.005170  22357.338806       103.5
        9     63930  22135.374968  22341.704604       108.4
        10    90906  22639.673093  22350.351077       112.7
        11   102738  24006.595669  22927.214577       114.4
        12    98063  23562.055686  23402.774816       114.3
   2020 1     80103  22753.581033  23440.744129       113.0
        2    108386  23475.046127  23263.560949       115.9
        3     72854  21999.410099  22742.679086       107.0
        4     66598  22388.499167  22620.985131       103.1
        5     87285  23255.627201  22547.845489       109.8

**② 부동산 소비자심리지수 등급 구분하기 (labeling)**

In [39]:
def Labeling(x):
    label = ''
    
    if x == '-':
        label = '-'
    
    else:
        x = float(x)
        
        if x < 95:
            label = '하강'
        elif x < 115:
            label = '보합'
        else:
            label = '상승'
    
    return label

In [40]:
data['등급'] = data['부동산 소비심리지수'].apply(lambda x: Labeling(x))

print(data.shape)
data.head()

(60, 5)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


실거래량         매매가평균      매매가 이동평균  부동산 소비심리지수  등급
시도 거래년도 거래월                                                   
전국 2015 6    95512  18508.644592  18178.764753       123.5  상승
        7    94339  18562.564554  18392.895905       129.4  상승
        8    79605  18846.474674  18639.227940       128.7  상승
        9    85899  18833.881894  18747.640374       126.4  상승
        10   98098  19121.626182  18933.994250       129.8  상승

In [41]:
data.to_csv('./data/부동산심리지수.csv', encoding='utf-8-sig', header=True, index=True)

---

## **Ⅲ. 순환변동치**

<i>**순환변동치 = 현재 매매가 - 매매가의 추세**</i><br>
순환변동치를 0을 기준으로 양이면 호경기, 음이면 불경기를 의미한다. <br>

In [42]:
data['매매가 순환변동치'] = data['매매가평균'] - data['매매가 이동평균']
data = data[['실거래량', '매매가평균', '매매가 이동평균', '매매가 순환변동치', '부동산 소비심리지수', '등급']]
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


실거래량         매매가평균      매매가 이동평균   매매가 순환변동치  부동산 소비심리지수  등급
시도 거래년도 거래월                                                               
전국 2015 6    95512  18508.644592  18178.764753  329.879840       123.5  상승
        7    94339  18562.564554  18392.895905  169.668650       129.4  상승
        8    79605  18846.474674  18639.227940  207.246734       128.7  상승
        9    85899  18833.881894  18747.640374   86.241519       126.4  상승
        10   98098  19121.626182  18933.994250  187.631932       129.8  상승

In [43]:
data.to_csv('./data/최종 데이터.csv', encoding='utf-8-sig', header=True, index=True)